In [1]:
import visa

In [2]:
rm = visa.ResourceManager('@py')
resources = rm.list_resources()
resources

('ASRL/dev/ttyUSB2::INSTR',
 'ASRL/dev/ttyUSB3::INSTR',
 'ASRL/dev/ttyUSB-ZStick-2G::INSTR',
 'ASRL/dev/ttyUSB1::INSTR',
 'ASRL/dev/ttyUSB0::INSTR',
 'ASRL/dev/ttyACM1::INSTR',
 'ASRL/dev/ttyACM0::INSTR',
 'ASRL/dev/ttyAMA0::INSTR',
 'USB0::6833::1416::DS1EU152500705\x00::0::INSTR')

In [4]:
from ds1000de import DS1000DE
scope=DS1000DE("DS1EU152500705")

r = scope.inst.query
w = scope.inst.write

In [8]:
from mhs5200 import MHS5200

ImportError: No module named 'mhs5200'

In [9]:
port = "/dev/ttyUSB0" # Serial port.
signal_gen = MHS5200(port=port)


NameError: name 'MHS5200' is not defined

In [ ]:
grid_t = 12
grid_v = 8
cycles = 10

In [ ]:
period = 1/chan1.frequency # Seconds
scale_val = period*cycles/grid_t

In [ ]:
w(":TIMEBASE:SCALE {}".format(scale_val))

In [ ]:
scale_pct = 0.8
scale = chan1.amplitude/scale_pct/grid_v
w(":CHAN1:SCALE {}".format(scale))

In [ ]:
w(":CHAN1:OFFSET 0")

In [ ]:
class Measure(object):
    def __init__(self, scope):
        self.scope = scope
        
    @property
    def inst(self):
        return self.scope.inst
    
    @property
    def _kw_(self):
        return self.__class__.__name__.upper()
    
    
    def _write_(self, cmd, arg=None):
        if arg is None:
            self.inst.write(":{}:{}".format(self._kw_, cmd))
        else:
            self.inst.write(":{}:{} {}".format(self._kw_, cmd, arg))
    
    def _query_(self, cmd):
        return self.inst.query(":{}:{}?".format(self._kw_, cmd))
    
    def clear(self):
        self._write_("clear")

In [ ]:
measurements = [
    'vpp',
    'vmax',
    'vmin',
    'vamplitude',
    'vtop',
    'vbase',
    'vaverage',
    'vrms',
    'overshoot',
    'preshoot',
    'frequency',
    'risetime',
    'falltime',
    'period',
    'pwidth',
    'nwidth',
    'pdutycycle',
    'ndutycycle',
    'pdelay',
    'ndelay',
]

In [ ]:
def measurement_factory(measurement):
    def measure(self):
        value_raw = self._query_(measurement)
        return float(value_raw)
    return measure

In [ ]:
for measurement in measurements:
    fcn = measurement_factory(measurement)
    prop = property(fcn)
    setattr(Measure, measurement, prop)

In [ ]:
m = Measure(scope)

In [ ]:
m.vmax

In [ ]:
m.frequency

In [ ]:
m.ndutycycle

In [ ]:
m.period

In [ ]:
chan1.amplitude=10

In [ ]:
m.vpp

In [ ]:
m.vamplitude

In [ ]:
def Q(*args):
    return r(":{}?".format(":".join(args)))
def W(*args):
    kw = ":".join(args[:-1])
    val = args[-1]
    return w(":{} {}".format(kw, val))

In [ ]:
Q("TRIGGER:MODE")

In [ ]:
Q("TRIGGER:EDGE:LEVEL")

In [ ]:
Q("TRIGGER:STATUS")

In [ ]:
Q("TRIGGER:EDGE:SWEEP")

In [ ]:
W("TRIGGER:EDGE:SWEEP", "SINGLE")

In [ ]:
m.vamplitude

In [ ]:
m.frequency

In [ ]:
m.period